In [ ]:
from tkinter import *
from tkinter import filedialog
import os
import numpy as np
# # Importing input

import os
root = Tk()
root.geometry("680x300")
root.title("Index")
noth = Label(root, text = "Processing...")
noth.config(font =("Courier", 12))
noth.pack()
my_filetypes = [('all files', '.*'), ('text files', '.txt')]
root.filename =  filedialog.askopenfilename(
                                    title="Please select a file:",
                                    filetypes=my_filetypes)
print(root.filename)
filename = os.path.basename(root.filename)
print(filename)
nameplox = os.path.splitext(filename)[0]
print(nameplox)  
nameplox = nameplox + "_summerized.txt"
print(nameplox)
text=open(root.filename, encoding="utf-8").read()



# # Sentence segmentation

from nltk import sent_tokenize
sentences=(sent_tokenize(text))
#print("Sentences:",sentences, end="\n")
#print(len(sentences))


emptyarray= np.empty((len(sentences),1,3),dtype=object)
for s in range(len(sentences)):
    emptyarray[s][0][0] = sentences[s]
    emptyarray[s][0][1] = s


# # Tokenization, Stop word removal , Bi-grams, Tri-grams

import nltk
from string import punctuation
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
bi_token=[]
bi_token_length=[]
tri_token_length=[]
for u in range(len(sentences)):
    sent_split1=[w.lower() for w in sentences[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
    
    bigrams_list = [bigram for bigram in nltk.bigrams(sent_split)]
    bi_token.append(bigrams_list)
    bi_token_length.append(len(bi_token[u]))
bi_tokens = [(int(o) / max(bi_token_length))*100 for o in bi_token_length]

tri_token=[]
for u in range(len(sentences)):
    sent_split2=[w.lower() for w in sentences[u].split(" ")]
    sent_split3=[w for w in sent_split2 if w not in stop_words and w not in punctuation and not w.isdigit()]
    trigrams_list = [trigram for trigram in nltk.trigrams(sent_split3)]
    tri_token.append(trigrams_list)
    tri_token_length.append(len(tri_token[u]))
tri_tokens = [(int(m) / max(tri_token_length))*100 for m in tri_token_length]

# # Sentence Position Feature

import math
def position(l):
    return [index for index, value in enumerate(sentences)]

sent_position= (position(sentences))
num_sent=len(sent_position)
#print("sentence position:",sent_position,end="\n")
#print("Total number of sentences:",num_sent,end="\n")
#th= 0.2*num_sent
#minv=th*num_sent
#maxv=th*2*num_sent
position = []
position_rbm = []
sent_pos1_rbm = 1
sent_pos1 = 100
position.append(sent_pos1)
position_rbm.append(sent_pos1_rbm)
for x in range(1,num_sent-1):
    #s_p = (math.cos((sent_position[x]-minv)*((1/maxv)-minv)))*100
    #if s_p < 0:
     #   s_p = 0
    s_p= ((num_sent-x)/num_sent)*100
    position.append(s_p)
    s_p_rbm = (num_sent-x)/num_sent
    position_rbm.append(s_p_rbm)
    
sent_pos2 = 100
sent_pos2_rbm = 1
position.append(sent_pos2)
position_rbm.append(sent_pos2_rbm)
#print("Sentence position feature vector:",position_rbm)

# # Converting Sentences to Vectors

def convertToVSM(sentences):
    vocabulary = []
    for sents in sentences:
        vocabulary.extend(sents)
    vocabulary = list(set(vocabulary))
    vectors = []
    for sents in sentences:
        vector = []
        for tokenss in vocabulary:
            vector.append(sents.count(tokenss))
        vectors.append(vector)
    return vectors
VSM=convertToVSM(sentences)
#print("SentenceVectors :: ",VSM)
#print("\n")

# # TF-ISF feature and Centroid Calculation

sentencelength=len(sentences)
def calcMeanTF_ISF(VSM, index):
    vocab_len = len(VSM[index])
    sentences_len = len(VSM)
    count = 0
    tfisf = 0
    for i in range(vocab_len):
        tf = VSM[index][i]
        if(tf>0):
            count += 1
            sent_freq = 0
            for j in range(sentences_len):
                if(VSM[j][i]>0): sent_freq += 1
            tfisf += (tf)*(1.0/sent_freq)
    if(count > 0):
        mean_tfisf = tfisf/count
    else:
        mean_tfisf = 0
    return tf, (1.0/sent_freq), mean_tfisf
tfvec=[]
isfvec=[]
tfisfvec=[]
tfisfvec_rbm=[]
for i in range(sentencelength):
    x,y,z=calcMeanTF_ISF(VSM,i)
    tfvec.append(x)
    isfvec.append(y)
    tfisfvec.append(z*100)
    tfisfvec_rbm.append(z)
maxtf_isf=max(tfisfvec_rbm)
centroid=[]
centroid.append(maxtf_isf)
centroid=(max(VSM))
#print(centroid)

# # Cosine Similarity between Centroid and Sentences

from numpy import dot
from numpy.linalg import norm
cosine_similarity=[]
cosine_similarity_rbm=[]
for z in range(sentencelength):
    cos_simi = ((dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))*100)
    cosine_similarity.append(cos_simi)
    cos_simi_rbm = (dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))
    cosine_similarity_rbm.append(cos_simi_rbm)
#print("Cosine Similarity Vector:",cosine_similarity_rbm)

# # Sentence length feature

sent_word=[]
for u in range(len(sentences)):
    sent_split1=[w.lower() for w in sentences[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
    a=(len(sent_split))
    sent_word.append(a)
#print("Number of words in each sentence:",sent_word)
#print("\n")

# # OR BY THIS METHOD: LENGTH OF SENTENCE/ LONGEST SENTENCE
longest_sent=max(sent_word)
sent_length=[]
sent_length_rbm=[]
for x in sent_word:
    sent_length.append((x/longest_sent)*100)
    sent_length_rbm.append(x/longest_sent)
    
# # Numeric token Feature

import re
num_word=[]
numeric_token=[]
numeric_token_rbm=[]
for u in range(len(sentences)):
    sent_split4=sentences[u].split(" ")
    e=re.findall("\d+",sentences[u])
    noofwords=(len(e))
    num_word.append(noofwords)
    numeric_token.append((num_word[u]/sent_word[u])*100)
    numeric_token_rbm.append(num_word[u]/sent_word[u])

# # Thematic words feature
from rake_nltk import Rake
r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
keywords=[]
# If you want to provide your own set of stop words and punctuations to
# r = Rake(<list of stopwords>, <string of puntuations to ignore>)

for s in sentences:
    r.extract_keywords_from_text(s)
    key=list(r.get_ranked_phrases())
    keywords.append(key)
#print(keywords)
l_keywords=[]
for s in keywords:
    leng=len(s)
    l_keywords.append(leng)
#print(l_keywords)

total_keywords=sum(l_keywords)
#print(total_keywords)

thematic_number= []
thematic_number_rbm= []
for x in l_keywords:
    thematic_number.append((x/total_keywords)*100)
    thematic_number_rbm.append(x/total_keywords)
#print("Thematic word feature", thematic_number_rbm)
#print("\n")

# # proper noun feature
from nltk.tag import pos_tag
from collections import Counter
pncounts = []
pncounts_rbm = []
for sentence in sentences:
    tagged=nltk.pos_tag(nltk.word_tokenize(str(sentence)))
    counts = Counter(tag for word,tag in tagged if tag.startswith('NNP') or tag.startswith('NNPS'))
    f=sum(counts.values())
    pncounts.append(f)
    pncounts_rbm.append(f)
pnounscore=[(int(o) / int(p))*100 for o,p in zip(pncounts, sent_word)]
pnounscore_rbm=[int(o) / int(p) for o,p in zip(pncounts_rbm, sent_word)]
#print(pncounts)
#print("Pronoun feature vector",pnounscore_rbm)
#print("\n")


# # feature matrix1

featureMatrix = []
featureMatrix.append(position_rbm)
featureMatrix.append(bi_token_length)
featureMatrix.append(tri_token_length)
featureMatrix.append(tfisfvec_rbm)
featureMatrix.append(cosine_similarity_rbm)
featureMatrix.append(thematic_number_rbm)
featureMatrix.append(sent_length_rbm)
featureMatrix.append(numeric_token_rbm)
featureMatrix.append(pnounscore_rbm)

featureMat = np.zeros((len(sentences),9))
for i in range(9) :
    for j in range(len(sentences)):
        featureMat[j][i] = featureMatrix[i][j]

#print("\n\n\nPrinting Feature Matrix : ")
#print(featureMat)
#print("\n\n\nPrinting Feature Matrix Normed : ")
#featureMat_normed = featureMat / featureMat.max(axis=0)
featureMat_normed = featureMat

#print(featureMat_normed)
for i in range(len(sentences)):
    #print(featureMat_normed[i])
    print("Done")
#np.save('output_labels_10.npy',featureMat_normed)

import rbm

temp = rbm.test_rbm(dataset = featureMat_normed,learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9,
             n_hidden=9)

#print("\n\n")
#print(np.sum(temp, axis=1))

enhanced_feature_sum = []

for i in range(len(np.sum(temp,axis=1))) :
    enhanced_feature_sum.append([np.sum(temp,axis=1)[i],i])
    emptyarray[i][0][2]=np.sum(temp,axis=1)[i]
#print("enhanced feature sum",enhanced_feature_sum)
#print("\n\n\n")

enhanced_feature_sum.sort(key=lambda x: x[0])
#print("enhanced feature sum sorted",enhanced_feature_sum)

length_to_be_extracted = len(enhanced_feature_sum)/2

#print("\n\nThe text is : \n\n")
for x in range(len(sentences)):
    #print(sentences[x])
    print("Done")
#print("\n\n\nExtracted sentences : \n\n\n")
extracted_sentences = []
extracted_sentences.append([sentences[0], 0])

for x in range(int(length_to_be_extracted)) :
    if(enhanced_feature_sum[x][1] != 0) :
        extracted_sentences.append([sentences[enhanced_feature_sum[x][1]], enhanced_feature_sum[x][1]])
       

    #print(extracted_sentences)
extracted_sentences.sort(key=lambda x: x[1])
#print("\n\n")
#print(extracted_sentences)


######array method to print summary############
#emparray1 = emptyarray[0]
#emparray2 = emptyarray[1:]
#emparray2 = emparray2[emparray2[:,:,1].argsort()]
#emparray3 = np.concatenate((emparray1,emparray2))
#print(emparray3)



#summary_file = open("gen_summary.txt","w")
summary1=[]
#finalText = ""
#print("\n\n\nExtracted Final Text : \n\n\n")
for i in range(len(extracted_sentences)):
    final_text="\n"+extracted_sentences[i][0]
    final_text_1=extracted_sentences[i][0]
    summary1.append(final_text_1)
    #print(final_text)
    #print(summary1)
    #print(len(summary1))
    #summary=" ".join(final_text)
    #print(summary)
    #finalText = finalText + extracted_sentences[i][0]

    #summary_file.write(final_text)
    #summary_file.close()

emparray1 = emptyarray[0]
#print(emparray1)
emparray2 = emptyarray[1:,:,:]

emparray2 = emparray2[np.argsort(emparray2[:,0,2])]
emparray2 = emparray2[::-1]
sh=emparray2.shape[0]
sh=int(sh/2)+1
emparray2=emparray2[:sh]
emparray2 = emparray2[np.argsort(emparray2[:,0,1])]
#print(emparray2)
#emparray3 = np.concatenate((emparray1,emparray2), axis=1)
#print(emparray3)

rbmarray=emparray2[:,:,:2]
rbm_summary = []
for i in range(rbmarray.shape[0]):
    rbm_summary.append(rbmarray[i][0][0])
    
#print("RBM summary \n\n",rbm_summary)
#print(rbmarray)
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# # New Antecedent/Consequent objects hold universe variables and membership
# # functions

position1 = ctrl.Antecedent(np.arange(0, 100, 10), 'position1')
cos_similarity = ctrl.Antecedent(np.arange(0, 100, 10), 'cos_similarity')
bitokens = ctrl.Antecedent(np.arange(0, 100, 10), 'bitokens')
tritokens = ctrl.Antecedent(np.arange(0, 100, 10), 'tritokens')
propernoun = ctrl.Antecedent(np.arange(0, 100, 10), 'propernoun')
sentencelength = ctrl.Antecedent(np.arange(0, 100, 10), 'sentencelength')
numtokens = ctrl.Antecedent(np.arange(0, 100, 10), 'numtokens')
keywords = ctrl.Antecedent(np.arange(0, 10, 1), 'keywords')
tf_isf = ctrl.Antecedent(np.arange(0, 100, 10), 'tf_isf')


senten = ctrl.Consequent(np.arange(0, 100, 10), 'senten')

position1.automf(3)
cos_similarity.automf(3)
bitokens.automf(3)
tritokens.automf(3)
propernoun.automf(3)
sentencelength.automf(3)
numtokens.automf(3)
keywords.automf(3)
tf_isf.automf(3)


senten['bad'] = fuzz.trimf(senten.universe, [0, 0, 50])
senten['avg'] = fuzz.trimf(senten.universe, [0, 50, 100])
senten['good'] = fuzz.trimf(senten.universe, [50, 100, 100])

rule1 = ctrl.Rule(position1['good'] & sentencelength['good'] & propernoun['good'] &numtokens['good'], senten['good'])
rule2 = ctrl.Rule(position1['poor'] & sentencelength['poor'] & numtokens['poor'], senten['bad'])
rule3 = ctrl.Rule(propernoun['poor'] & keywords['average'], senten['bad'])
rule4 = ctrl.Rule(cos_similarity['good'], senten['good'])
rule5 = ctrl.Rule(bitokens['good'] & tritokens['good'] & numtokens['average'] | tf_isf['average'], senten['avg'])


sent_ctrl = ctrl.ControlSystem([rule1,rule2,rule3,rule4,rule5])
Sent = ctrl.ControlSystemSimulation(sent_ctrl)
fuzzemptyarr= np.empty((20,1,2), dtype=object)
t2=0
summary2=[]
for s in range(len(sentences)):
    Sent.input['position1'] = int(position[s])
    Sent.input['cos_similarity'] = int(cosine_similarity[s])
    Sent.input['bitokens'] = int(bi_tokens[s])
    Sent.input['tritokens'] = int(tri_tokens[s])
    Sent.input['tf_isf'] = int(tfisfvec[s])
    Sent.input['keywords'] = int(thematic_number[s])
    Sent.input['propernoun'] = int(pnounscore[s])
    Sent.input['sentencelength'] = int(sent_length[s])
    Sent.input['numtokens'] = int(numeric_token[s])
#Sent.input['service'] = 2
    Sent.compute()
    if Sent.output['senten'] > 80:
        summary2.append((sentences[s]))
        fuzzemptyarr[t2][0][0] = sentences[s]
        fuzzemptyarr[t2][0][1] = s
        t2+=1
fuzzarray = np.empty((len(summary2),1,2),dtype=object)
for i in range(len(summary2)):
    fuzzarray[i][0][0] = fuzzemptyarr[i][0][0]
    fuzzarray[i][0][1] = fuzzemptyarr[i][0][1]
    
fuzzarray=fuzzarray[1:]
#print("Fuzzy logic summary \n\n",summary2)
#print(len(summary2))
#print(fuzzarray)
    #senten.view(sim=Sent)

    
summarray=np.empty((emparray2.shape), dtype=object) 
t1=0
for i in range(emparray2.shape[0]):
    t = emparray2[i][0][0]
    t2 =emparray2[i][0][1]
    for j in summary2:
        if t == j:
            summarray[t1][0][0] = t
            summarray[t1][0][1] = t2
            t1 += 1
commsentarray=summarray[summarray != np.array(None)]
commonarray = np.empty((int(commsentarray.shape[0]/2),1,2),dtype=object)
for i in range((int(commsentarray.shape[0]/2))):
    commonarray[i][0][0] = summarray[i][0][0]
    commonarray[i][0][1] = summarray[i][0][1]
#print(commonarray)
 
concarray=np.concatenate((rbmarray,fuzzarray), axis=0)
#print(concarray.shape)
#print(rbmarray.shape)
#print(fuzzarray.shape)

concarray2 = concarray

for i in range(commonarray.shape[0]):
    t1 = commonarray[i][0][0]
    for j in range(concarray.shape[0]):
        t2 = concarray[j][0][0]
        if t1 == t2:
            concarray2[j][0][0] = 0
            concarray2[j][0][1] = 0
            
#print(concarray2)
concarray3=concarray2[concarray2 != 0]
#print(concarray3)
#print(len(concarray3))
uncommonarray= np.empty((int(concarray3.shape[0]/2),1,2), dtype=object)
t1=0
t2=0

for i in range(concarray3.shape[0]):
    if i % 2 == 0:
        uncommonarray[t1][0][0]=concarray3[i]
        t1+=1
    else:
        uncommonarray[t2][0][1]=concarray3[i]
        t2+=1
#print(uncommonarray)
#print(uncommonarray.shape)   
uncommonarray=uncommonarray[np.argsort(uncommonarray[:,0,1])]
uncommonarray=uncommonarray[:int(uncommonarray.shape[0]/2)]
#print(uncommonarray)


######### including first sentence ###########
emparray5=emparray1[0][0]
firstsent=np.empty((1,1,2),dtype=object)
emparray6=emparray1[0][1]
firstsent[0][0][0]=emparray5
firstsent[0][0][1]=emparray6
#print(firstsent)
#print(firstsent.shape)
#print(commonarray.shape)
#print(uncommonarray.shape)

########## final summary ############
final_summ_array=np.concatenate((firstsent,commonarray,uncommonarray))
final_summ_array=final_summ_array[np.argsort(final_summ_array[:,0,1])]
#print(final_summ_array)

final_summary = []
for i in range(final_summ_array.shape[0]):
    final_summary.append(final_summ_array[i][0][0])
    final = " ".join(final_summary)
print(final)

full_text = text
summarized_text = final
file = open(nameplox,"w+")
file.write(summarized_text)
file.close()

def summery():
    from tkinter import font
    import tkinter.messagebox

    swami = Tk()
    swami.title("Save summerized file")
    swami.geometry("1200x660")

    l = Label(swami, text = "Summerized Text")
    l.config(font =("Courier", 12))
    l.pack()

    myframe= Frame(swami)
    myframe.pack(pady=5)

    text_scroll = Scrollbar(myframe)
    text_scroll.pack(side=RIGHT, fill=Y)

    mytext=Text(myframe, width=97,height=25,font=("Helvetica", 16), undo=True, yscrollcommand=text_scroll.set)
    mytext.insert(INSERT, summarized_text)
    mytext.pack()

    text_scroll.config(command=mytext.yview)

    mymenu = Menu(swami)
    swami.config(menu=mymenu)

    def onClick():
        tkinter.messagebox.showinfo("Already Saved",  "Already saved as "+ nameplox)

    filemenu = Menu(mymenu, tearoff=False)
    mymenu.add_cascade(label="File", menu=filemenu)
    filemenu.add_command(label="Save",command=onClick)
    filemenu.add_separator()
    filemenu.add_command(label="Exit", command=swami.destroy)
    swami.mainloop()

import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

def get_nouns_multipartite(text):
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text)
    #    not contain punctuation marks or stopwords as candidates.
    pos = {'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos, stoplist=stoplist)

    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out

keywords = get_nouns_multipartite(full_text) 
print (keywords)
filtered_keys=[]
for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
        
print (filtered_keys)

from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
print (keyword_sentence_mapping)

import requests
import json
import re
import random
import nltk
import pywsd
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn

def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
    
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
                                                   
def closeit():
    try:
        nathan.destroy()
    except:
        print("already closed")
    root.destroy()
from tkinter import *
nathan = Tk()
nathan.title("Question Generation")
nathan.geometry("1200x660")
l = Label(nathan, text = "MCQ and FILL-ME")
l.config(font =("Courier", 12))
l.pack()
myframe= Frame(nathan)
myframe.pack(pady=5)
text_scroll = Scrollbar(myframe)
text_scroll.pack(side=RIGHT, fill=Y)
mytext=Text(myframe, width=97,height=25,font=("Helvetica", 16), undo=True, yscrollcommand=text_scroll.set)

index = 1
mytext.insert(INSERT, "#############******##############")
mytext.pack()
mytext.insert(INSERT, "This is MCQ Pattern Questions")
mytext.pack()
mytext.insert(INSERT, "#############******##############")
mytext.pack()
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    two = ("%s)"%(index),output)
    mytext.insert(INSERT, "\n\n")
    mytext.pack()
    mytext.insert(INSERT, two)
    mytext.pack()
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        three = (optionchoices[idx], choice)
        mytext.insert(INSERT, "\n")
        mytext.pack()
        mytext.insert(INSERT, optionchoices[idx])
        mytext.pack()
        mytext.insert(INSERT, ") "+choice)
        mytext.pack()
    #print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1

mytext.insert(INSERT, "\n\n #############******##############")
mytext.pack()
mytext.insert(INSERT, "This is Fill me Pattern Questions")
mytext.pack()
mytext.insert(INSERT, "#############******##############")
mytext.pack()
for each in keyword_sentence_mapping:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    one = ("%s)"%(index),output)
    mytext.insert(INSERT, "\n\n")
    mytext.pack()
    mytext.insert(INSERT, one)
    mytext.pack()
    choice = each
    mytext.insert(INSERT, "\n\nAnswer : " + each)
    mytext.pack()
    index = index + 1
print("backend finished")
text_scroll.config(command=mytext.yview)
mymenu = Menu(nathan)
nathan.config(menu=mymenu)
Butt = Button(nathan, text ="Hide", command = nathan.withdraw)
Butt.pack()
print("half over")
noth.config(text = "Work done select any")
buttons = Button(root, text ="Close", command = closeit)
buttons.pack(padx=20, pady=20)
button = Button(root, text ="Summary", command = summery)
button.pack(padx=20, pady=20)
butto = Button(root, text ="Questions", command = nathan.deiconify)
butto.pack(padx=20, pady=20)
king = Label(root, text = "Hide MCQ window don't close or else need to re-run for open it")
king.config(font =("Courier", 12))
king.pack()
print("ended")
nathan.withdraw()
nathan.mainloop()
